#  ENGR7761 Computer vision project

## Load the video and libraries

In [11]:
import cv2
import numpy as np

# Load the video
cap = cv2.VideoCapture('SlowMotionDartsThrow.mp4')
# Check if camera opened successfully
if not cap.isOpened(): 
    print("Error opening video stream or file")

# Read the first frame to get its shape
ret, frame = cap.read()
if not ret:
    print("Can't receive frame (stream end?). Exiting ...")
    exit()

while True:
    flag, img = cap.read()
    
    if not flag: # Check if there is data in the frame
        print("End of video. Exiting ...")
        break
        
    cv2.imshow("Default video", img)
    # Press 'q' on keyboard to exit
    if cv2.waitKey(10) == ord('q'):
        break
# Release everything if job is finished
cap.release()
cv2.destroyAllWindows()

End of video. Exiting ...


## Apply Gaussian blur to the video

In [14]:
import cv2
import numpy as np

# Load the video
cap = cv2.VideoCapture('SlowMotionDartsThrow.mp4')
# Check if camera opened successfully
if not cap.isOpened(): 
    print("Error opening video stream or file")

# Read the first frame to get its shape
ret, frame = cap.read()
if not ret:
    print("Can't receive frame (stream end?). Exiting ...")
    exit()

""" Saving the modified video """
frame_height, frame_width = frame.shape[:2] # Size to save as (same as input)
fps = 30.0 # Frames per second
fourcc = cv2.VideoWriter_fourcc(*'H264') # H264 video format
out = cv2.VideoWriter('BLurredVideo.mp4', fourcc, fps, (frame_width, frame_height)) # Output file name
    
    
# Blur parameters
kernel_size = 21

    
while True:
    flag, img = cap.read()
    
    if not flag: # Check if there is data in the frame
        print("End of video. Exiting ...")
        break
    blurred = cv2.GaussianBlur(img, (kernel_size, kernel_size), 10)
    output_image = blurred.copy()
    
    # Save the frame into the 'BlurredVideo.mp4' file
    out.write(output_image)
    
    # Display on the video
    cv2.imshow("Blurred video", output_image)
    
    # Press 'q' on keyboard to exit
    if cv2.waitKey(10) == ord('q'):
        break
# Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()

End of video. Exiting ...


## Apply average filter to the blurred video

In [16]:
import cv2
import numpy as np

# Load the video
cap = cv2.VideoCapture('BLurredVideo.mp4')
# Check if camera opened successfully
if not cap.isOpened(): 
    print("Error opening video stream or file")

# Read the first frame to get its shape
ret, frame = cap.read()
if not ret:
    print("Can't receive frame (stream end?). Exiting ...")
    exit()

""" Saving the modified video """ 
frame_height, frame_width = frame.shape[:2] # Size to save as (same as input)
fps = 30.0 # Frames per second
fourcc = cv2.VideoWriter_fourcc(*'H264') # H264 video format
out = cv2.VideoWriter('MovingAvgBLurredVideo.mp4', fourcc, fps, (frame_width, frame_height)) # Output file name
    
    
# Processing parameters
fbuffer = 13
alpha = float(1.0 / fbuffer)
difference_thresh = 4
flag, img = cap.read()
if flag:
    movingaverage = np.float32(img)
while True:
    flag, img = cap.read()
    
    if not flag: # Check if there is data in the frame
        print("End of video. Exiting ...")
        break
    
    # Calculated the weigthed average of the currrent frame
    cv2.accumulateWeighted(img, movingaverage, alpha)
    res = cv2.convertScaleAbs(movingaverage)
    difference_img = cv2.absdiff(res, img)
    
    # Perform binary threshold
    grey_difference_img = cv2.cvtColor(difference_img, cv2.COLOR_BGR2GRAY) # Convert to greyscale
    ret, motionmask = cv2.threshold(grey_difference_img, difference_thresh, 255, cv2.THRESH_BINARY)
    
    # 
    output_image = motionmask.copy()
    
    # Save the frame into the 'MovingAvgBLurredVideo.mp4' file
    out.write(output_image)
    
    # Display on the video
    cv2.imshow("Moving average blurred video", output_image)
    
    # Press 'q' on keyboard to exit
    if cv2.waitKey(10) == ord('q'):
        break
# Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()

End of video. Exiting ...


## Apply Kanade–Lucas–Tomasi feature tracker to the pre-processed video

In [18]:
import cv2
import numpy as np

# Load the video
cap = cv2.VideoCapture('MovingAvgBLurredVideo.mp4') # Load the first video
cap2 = cv2.VideoCapture('SlowMotionDartsThrow.mp4') # Load the second video

# Check if video works opened successfully
if not cap.isOpened(): 
    print("Error opening video stream or file")
if not cap2.isOpened(): 
    print("Error opening video stream or file")

# Read the first frame to get its shape
ret, frame = cap.read() # Read the first frame of the first video
if not ret:
    print("Can't receive frame (stream end?). Exiting ...") # Print error message if the first frame couldn't be read
    exit()
# Read the first frame to get its shape
ret2, frame2 = cap2.read() # Read the first frame of the second video
if not ret2:
    print("Can't receive frame (stream end?). Exiting ...") # Print error message if the first frame couldn't be read
    exit()

# Saving the modified video
frame_height, frame_width = frame.shape[:2] # Get the height and width of the frame
fps = 30.0 # Set the frames per second
fourcc = cv2.VideoWriter_fourcc(*'H264') # Set the video format
out = cv2.VideoWriter('KLTTrackedDart.mp4', fourcc, fps, (frame_width, frame_height)) # Create a VideoWriter object for the first output video
out2 = cv2.VideoWriter('KLTTrackedDartOverlayOriginal.mp4', fourcc, fps, (frame_width, frame_height)) # Create a VideoWriter object for the second output video

# Parameters for ShiTomasi corner detection
feature_params = dict(maxCorners = 3, qualityLevel = 0.3, minDistance = 50, blockSize = 5) # Set the parameters for ShiTomasi corner detection

# Parameters for Lucas-Kanade optical flow
WindowSize = (25, 25) # Set the window size
lk_params = dict(winSize = WindowSize, maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 20, 0.03)) # Set the parameters for Lucas-Kanade optical flow

# Create a mask image for drawing purposes
mask = None

# Initialize old_gray variable
old_gray = None
    
while True:
    ret, frame = cap.read() # Read the next frame from the first video
    ret2, frame2 = cap2.read() # Read the next frame from the second video
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...") # Print error message if the frame couldn't be read
        break
    if not ret2:
        print("Can't receive frame (stream end?). Exiting ...") # Print error message if the frame couldn't be read
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # Convert the frame to grayscale

    # If this is the first frame or no good points were found in the last frame
    if mask is None or good_new.size == 0:
        p0 = cv2.goodFeaturesToTrack(frame_gray, mask = None, **feature_params) # Detect good features to track
        if p0 is None:
            continue
        mask = np.zeros_like(frame) # Create a mask of zeros with the same size as the frame

    # If this is the first frame, initialize old_gray
    if old_gray is None:
        old_gray = frame_gray.copy() # Copy the current grayscale frame to old_gray

    # Calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params) # Calculate the optical flow

    # Select good points
    good_new = p1[st==1] # Select the new points where the status is 1
    good_old = p0[st==1] # Select the old points where the status is 1

    # Draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)): # For each pair of new and old points
        a,b = map(int, new.ravel()) # Get the coordinates of the new point
        c,d = map(int, old.ravel()) # Get the coordinates of the old point
        mask = cv2.line(mask, (a,b),(c,d), (0,255,0), 2) # Draw a line from the old point to the new point on the mask
        frame = cv2.circle(frame,(a,b),5,(0,0,255),-1) # Draw a circle at the new point on the frame
        
    # Mask the image to the frame
    OriginalMask = cv2.add(frame2,mask) # Add the mask to the orignal frame
    out2.write(OriginalMask) # Write the result to the second output video
    img = cv2.add(frame,mask) # Add the mask to the motion thing frame
    output_image =img.copy() # Copy the result
    out.write(output_image) # Write the frame to vid
    cv2.imshow('frame',img) # Print 
    k = cv2.waitKey(15) & 0xff # Wait for a key press
    if k == 27: # If the ESC key was pressed
        break # Break the loop
    
    # Update frames
    old_gray = frame_gray.copy() # Copy the current grayscale frame to old_gray
    p0 = good_new.reshape(-1,1,2) # Reshape good_new and assign it to p0

cv2.destroyAllWindows() # Destroy all windows
out.release() # Stop recording
out2.release() # Stop recording
cap.release() # Release the first VideoCapture object
cap2.release() # Release the 2nd VideoCapture object

Can't receive frame (stream end?). Exiting ...
